# Наивный пример формирования рекомендации

Идея рекомендательных систем на основе ближайшего соседа – использовать предпочтения некоторой группы людей, похожих на данного пользователя, чтобы рекомендовать ему что-либо.

У такой модели много применений:
    
   * рекомендация товаров интернет-магазинов
   * рекомендация музыки или фильмов
   * рекомендация людей/контента в социальных сетях

Естественный способ получить рекомендацию о чем-либо - спросить мнение об этом у друзей или любых людей, которым нравится то же, что и вам. Эту идею можно использовать и для машины: для каждого человека алгоритм просматривает большую группу людей и ищет в ней подгруппу с похожим на данного человека вкусом. Далее создается список того, что еще нравится этим людям, а затем человеку рекомендуются предложения из этого списка. Такой алгоритм называется алгоритмом **коллаборативной фильтрации**.

**Отключаем предупреждения**

Библиотека **warnings** отвечает за то, какие предупреждения (warnings) о работе будут выводиться пользователю.
FutureWarning - предупреждения о том, как изменится работа библиотек в будущих версиях. Такие предупреждения мы будем игнорировать.
Чтобы включить режим игнорирования, мы отбираем все предупреждения из категории FutureWarning и выбираем для них действия 'ignore'.
Это делается вызовом функции simplefilter c задание двух атрибутов: действия action и категории предупреждений category.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

Для красивого вывода на экран сложных объектов будем использовать **pprint**

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

### Подготовка данных

Чтобы хранить сразу много предпочтений для каждого человека, удобнее всего воспользоваться **вложенным словарем**.


```python
dict = {'key': 'value'}
```

Мы будем работать с *вложенным словарем* кинокритиков и выставленных ими оценок для небольшого набора данных о фильмах. Построим его следующим образом:

1) для каждого кинокритика создаем словарь оценок фильмов в формате
   ```python
    scores_dict = {
        'film_1': 'score_1',
        'film_2': 'score_2',
        ...
    }
   ```
   
2) создаем словарь кинокритиков, где в качестве значений будет соответствующий ему словарь оценок
   ```python
     critics = {
         'name_1': 'scores_dict_1',
         'name_2': 'scores_dict_2',
         ...
     }
   ```
   
**Code 1:**

In [ ]:
# Словарь кинокритиков и выставленных ими оценок
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5,
 'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5,
 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 3.5},
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
 'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
 'The Night Listener': 4.5, 'Superman Returns': 4.0,
 'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 2.0},
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

In [ ]:
pp.pprint(critics)

{   'Claudia Puig': {   'Just My Luck': 3.0,
                        'Snakes on a Plane': 3.5,
                        'Superman Returns': 4.0,
                        'The Night Listener': 4.5,
                        'You, Me and Dupree': 2.5},
    'Gene Seymour': {   'Just My Luck': 1.5,
                        'Lady in the Water': 3.0,
                        'Snakes on a Plane': 3.5,
                        'Superman Returns': 5.0,
                        'The Night Listener': 3.0,
                        'You, Me and Dupree': 3.5},
    'Jack Matthews': {   'Lady in the Water': 3.0,
                         'Snakes on a Plane': 4.0,
                         'Superman Returns': 5.0,
                         'The Night Listener': 3.0,
                         'You, Me and Dupree': 3.5},
    'Lisa Rose': {   'Just My Luck': 3.0,
                     'Lady in the Water': 2.5,
                     'Snakes on a Plane': 3.5,
                     'Superman Returns': 3.5,
                 

In [ ]:
for i in critics.keys():
  print(critics[i])

{'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 'The Night Listener': 3.0}
{'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 3.5}
{'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0, 'Superman Returns': 3.5, 'The Night Listener': 4.0}
{'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'The Night Listener': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 2.5}
{'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 2.0}
{'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5}
{'Snakes on a Plane': 4.5, 'You, Me and Dupree': 1.0, 'Superman Returns': 4.0}


**Code 2:**  
Выведем оценки, которые выставил Toby

In [ ]:
critics['Toby']

{'Snakes on a Plane': 4.5, 'You, Me and Dupree': 1.0, 'Superman Returns': 4.0}

**Code 3:**  
Выведем оценку критика Lisa Rose для фильма Lady in the Water

In [ ]:
critics['Lisa Rose']['Lady in the Water']

2.5

### Критерий похожести

Как мы уже говорили, чтобы выделить подгруппу людей с похожим вкусом, необходимо как-то определить, насколько люди похожи. В нашем случае мы будем сравнивать оценки людей у одинаковых фильмов. Мы рассмотрим два способа сравнения. С помощью
- расстояния Евклида
- корреляции Пирсона

**Оценка по евклидову расстоянию**

Евклидово расстояние для точек $x = (x_1, x_2, ... , x_n)$ и $y = (y_1, y_2, ... , y_n)$ определяется следующим образом:

\begin{equation}
d(x, y) = \sqrt{\sum_{i = 1}^n (x_i - y_i)^2}
\end{equation}

Для понимания расстояния Евклида рассмотрим случай, когда *n = 2*. То есть каждый объект выборки (точка) описывается двумя параметрами (координатами). $x = (x_1, x_2)$ и $y = (y_1, y_2)$. Тогда Евклидово расстояние равно расстоянию между точками на плоскости.

\begin{equation}
d(x, y) = \sqrt{\sum_{i = 1}^n (x_i - y_i)^2} = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}
\end{equation}



Для вычисления квадратного корня можно воспользоваться готовой функцией **sqrt()** из библиотеки для работы с числами **math**:

**Code 4:**

**Упражнение 1**  
Реализуем функцию для вычисления расстояния Евклида

In [ ]:
from math import sqrt
import math
import numpy as np

def compute_euclid(x, y):
  p_1 = np.asarray(x)
  p_2 = np.asarray(y)
  return np.sqrt(sum((p_1 - p_2)**2) )

def compute_euclid(x, y):
  return math.dist(x,y)

Для примера вычислим расстояния Евклида для близких точек с координатам (1, 1) и (2, 1):

**Code 5:**

In [ ]:
compute_euclid([1, 1, 1], [1, 1, 2])

1.0

Теперь вычислим расстояния Евклида для точек, которые находятся далеко (1, 1) и (8, 10):

**Code 6:**

In [ ]:
compute_euclid([1, 1], [8, 10])

11.40175425099138

Расстояние, вычисленное по этой формуле, будет тем меньше, чем больше сходство людей (чем ближе точки). Нам же нужна функция, значение которой будет, наоборот, большое, если люди сильно похожи друг на друга. То есть схожесть больше (от 0 до 1), если точки ближе друг другу


Для этого будем использовать функцию "похожести" в таком виде:

\begin{equation}
\text{similarity }(x, y) = \dfrac{1}{1 + d(x, y)},
\end{equation}

где $d(x, y)$ - расстояние Евклида.

Пример вычисления сходства по новой функции для точек с координатами (1, 1) и (2, 3):

**Code 7:**

In [ ]:
1 / (1 + compute_euclid([1, 1], [2, 3]))

0.3090169943749474

Пример вычисления сходства по новой функции для точек с координатами (1, 1) и (8, 10):

**Code 8:**

In [ ]:
1 / (1 + compute_euclid([1, 1], [8, 10]))

0.08063375388365411

Далее реализуем несколько простых функций для коллаборативной фильтрации на основе ближайшего соседа.

**Code 9:**

**Упражнение 2.** Получить список фильмов, оцененных обоими

In [ ]:
def similar_films(critics_dict, person1, person2):
    # создаем пустой список фильмов с названием
    sim_film = []
    # для всех фильмов у person1, если такой фильм оценивался person 2, добавляем этот фильм в список sim_film
    films_person_1 = critics_dict[person1]
    films_person_2 = critics_dict[person2]
    for film in films_person_1:
      if film in films_person_2.keys():
        sim_film.append(film)

    return sim_film

Сначала реализуем функцию для подсчета сходства двух критиков на основе расстояния Евклида. Назовём функцию  **sim_distance()**. Функция будет использовать функцию **similar_films()** и принимать на вход три аргумента:
* словарь критиков
* имя первого критика для сравнения
* имя второго критика для сравнения

**Упражнение 3**. Подсчет сходства двух критиков на основе расстояния Евклида

In [ ]:
def sim_distance(critics_dict, person1, person2):
    # используем написанную функцию для получения фильмов, оцененных обоими критиками
    sim_films = similar_films(critics_dict, person1, person2)

    # если нет ни одной общей оценки, то есть длина sim_film равна 0 - вернуть 0
    if len(sim_films) == 0:
        return 0

    # посчитаем евклидово расстояние между критиками по фильмам из sim_films

    # посчитаем похожесть score

    films_person_1 = critics_dict[person1]
    films_person_2 = critics_dict[person2]

    score_person_1 = [films_person_1[film] for film in sim_films]
    score_person_2 = [films_person_2[film] for film in sim_films]
    dist = math.dist(score_person_1, score_person_2)
    score = 1 / (1 + dist)

    return score

Посмотрим, как работает эта функция для двух критиков Lisa Rose и Toby:

**Code 10:**

In [ ]:
sim_distance(critics, 'Lisa Rose', 'Toby')

0.3483314773547883

Эта функция работает также как:

\begin{equation}
\text{similarity }(x, y) = \dfrac{1}{1 + d(x, y)},
\end{equation}

где $d(x, y)$ - расстояние Евклида.

Однако, по осям будет фильмы и оценки от критиков. Если у критиков нет одинаковых фильмов, то мы этих критиков не рассматриваем

**Коэффициент корреляции Пирсона**

Другой способ определять степень схожести - использовать коэффициент корреляции Пирсона. Этот коэффициент показывает, насколько два набора данных хорошо ложатся на прямую.

Два кинокритика со средним значением корреляции 0.41 (слева) и два кинокритика с высоким значением корреляции 0.75 (справа).  

Корреляция Пирсона считается по формуле:
    
\begin{equation}
    \text{corr}(x, y) = \dfrac{\sum_{i = 1}^n (x_i - \frac{\sum_{j = 1}^n x_j}{n}) (y_i - \frac{\sum_{j = 1}^n y_j}{n})}{\sqrt{\sum_{i = 1}^n (x_i - \frac{\sum_{j = 1}^n x_j}{n})^2 \sum_{i = 1}^n (y_i - \frac{\sum_{j = 1}^n y_j}{n})^2}}
\end{equation}

По аналогии с функцией **sim_distance()** реализуйте функцию для подсчета сходства двух критиков на основе корреляции Пирсона. Назовём её  **sim_pearson()**. Функция принимает на вход те же самые аргументы, что и **sim_distance()**:
* словарь критиков
* имя первого критика для сравнения
* имя второго критика для сравнения

Посмотрим, как работает эта функция для двух критиков Lisa Rose и Gene Seymour:

**Code 11:**  
**Упражнение 4.** Допишите функцию для вычисления корреляции Пирсона

In [ ]:
def pearson_corr(x, y):
  p_1 = np.asarray(x)
  p_2 = np.asarray(y)
  cov = np.sum(
      (p_1 - np.mean(p_1))*(p_2 - np.mean(p_2))
      )
  sigma_1 = np.std(p_1, ddof=1)
  sigma_2 = np.std(p_2, ddof=1)

  return cov / (sigma_1 * sigma_2*(len(p_1) - 1))

Это та же функция, но из библиотеки

In [ ]:
from scipy.stats import pearsonr


x = [1,2,3,4]
y = [4,5,6,3]

assert pearsonr(x, y)[0] == pearson_corr(x, y), "Что-то не так"

**Упражнение 5.** Допишите функцию **sim_pearson()**

In [ ]:
# Возвращает коэффициент корреляции Пирсона между person1 и person2
def sim_pearson(critics_dict, person1, person2):
    from math import isnan # это нам пригодиться для обработки деления на 0

    # получить список фильмов, оцененных обоими критиками
    sim_films = similar_films(critics_dict, person1, person2)

    ## your code here

    films_person_1 = critics_dict[person1]
    films_person_2 = critics_dict[person2]

    score_person_1 = [films_person_1[film] for film in sim_films]
    score_person_2 = [films_person_2[film] for film in sim_films]
    r = pearsonr(score_person_1, score_person_2)[0]

    ## end of your code

    # если вдруг было деление на ноль и функция вернула nan, то присваиваем ноль
    if isnan(r):
        r = 0
    return r

In [ ]:
sim_pearson(critics, 'Lisa Rose', 'Gene Seymour')

0.3960590171906697

### Ранжирование критиков

После того как мы выбрали критерий для сравнения двух людей, можно начать искать для каждого человека подгруппу других людей с наиболее похожими вкусами.

Для этого реализуем функцию **topMatches()**. Функция будет возвращать список n наилучших соответствий для человека из словаря `critics`.

Соответственно, аргументы у функции:
* словарь критиков *critics\_dict*
* имя персоны *person*, для которой подбираем соответствия
* число *n* наилучших соответствий

Кроме этого, так как мы хотим использовать различные расстояния, то в качестве аргумента мы будем также передавать имя функции-расстояния (*similarity*).  


**Code 12:**

Как работает эта функция:
*  считаем схожесть интересующего нас критика с каждым другим критиком используя функцию **similarity**
*  сортируем результаты
*  возвращаем топ **n** результатов


**Упражнение 6**. Напишите функцию, которая возвращает топ-n наиболее похожих на person человек из critics_dict

In [ ]:
def topMatches(critics_dict, person, n=5, similarity=sim_pearson):
    critics_dict = critics_dict.copy()
    critics_dict_all = critics_dict.copy()
    del critics_dict[person]

    scores = {}
    for name in critics_dict:
      score = similarity(critics_dict_all, name, person)
      scores[name] = score

    scores = dict(sorted(scores.items(), reverse=True, key=lambda x: x[1]))
    return list(scores.keys())[:n]


    # инициализируем лист оценок похожести
    # для каждого человека в словаре критиков, если этот человек не тот, для которого мы подбираем соответствия
    # добавить в scores похожесть, вычисленную с помощью функции, записанной в similarity, и имя человека
    # отсортировать список по убыванию оценок
    # возвращаем первые n человек

**Code 13:**  
**Упражнение 7.** Вычислите топ-3 критиков похожих на Toby по евклидову расстоянию и по корреляции Пирсона. Они отличаются?

In [ ]:
topMatches(critics, 'Toby', 3, similarity=sim_distance)

['Mick LaSalle', 'Michael Phillips', 'Claudia Puig']

In [ ]:
topMatches(critics, 'Toby', 3, similarity=sim_pearson)

['Lisa Rose', 'Mick LaSalle', 'Claudia Puig']

Они различны, поскольку корреляция показывает линейнозависимость векторов, а Евклидово расстояние просто показывает расстояние между объектами. Если мы хотим делать задачу рекомендаций, то корреляция Пирсона здесь подходит лучше

Выбор расстояния для подбора похожести зависит от конкретной задачи. Имеет смысл попробовать разные критерии и посмотреть, какой из них дает наилучший результат.

## Рекомендация фильмов (User-based подход)

Мы умеем находить людей с похожим мнением как у рассматриваемого человека. Но как получить конкретную рекомендацию с набором фильмов? Можно было бы просто посмотреть на то, какие фильмы понравились похожему человеку, и выбрать из них не просмотренные. Однако, такой способ не стабилен: можем, например, отобрать критика, которому по каким-то причинам понравился фильм, получивший негативные оценки от остальных.

Чтобы разрешить эту проблему, можно ранжировать сами фильмы, вычисляя взвешенную сумму оценок критиков. Мы будем брать оценку фильма у критика и умножать его на коэффициент похожести с тем, для которого мы строим рекомендацию. Затем полученный результат суммировать и нормировать.  

Идея такого подхода в том, что люди похожие на тебя вносят больший вклад в "рейтинг" фильма.

Давайте реализуем этот подход в функции **getRecommendations()**. Функция принимает на вход следующие аргументы:
* словарь критиков *critics\_dict*
* имя персоны *person*, для которой подбираем соответствия
* имя функции-критерия для сравнения


Как работает эта функция:
*  считаем схожесть пользователя с каждым другим пользователем
*  перебираем фильмы, которые смотрели другие пользователи, но не смотрел текущий пользователь
*  нормируем рейтинг фильмов на основе того насколько он понравился другим пользователям, и того насколько каждый из других пользователей похож на текущего пользователя
*  сортируем список

**Code 14:**

In [ ]:
def getRecommendations(critics_dict, person, similarity=sim_pearson):
    critics_dict_all = critics_dict.copy()
    critics_dict = critics_dict.copy()
    films_person = critics_dict_all[person]

    del critics_dict[person]

    similar_people = topMatches(critics_dict_all, person, 100, similarity)

    coef = {name: similarity(critics_dict_all, name, person) for name in similar_people}

    films = set()
    for name in similar_people:
        films.update(critics_dict_all[name].keys())

    films = [film for film in films if film not in films_person]

    recomend_film = {}

    for film in films:
        weighted_scores = 0.0
        total_similarity = 0.0

        for name in similar_people:
            if film in critics_dict_all[name]:
                score = critics_dict_all[name][film]
                weighted_scores += score * coef[name]
                total_similarity += coef[name]

        if total_similarity > 0:
            recomend_film[film] = weighted_scores / total_similarity

    recomend_film = dict(sorted(recomend_film.items(), reverse=True, key=lambda x: x[1]))
    return recomend_film

**Code 15:**

In [ ]:
getRecommendations(critics,'Toby', sim_pearson)

{'The Night Listener': 3.119201586785551,
 'Lady in the Water': 3.002234730607126,
 'Just My Luck': 2.5309807037655645}

In [ ]:
getRecommendations(critics,'Toby', sim_distance)

{'The Night Listener': 3.457128694491423,
 'Lady in the Water': 2.778584003814924,
 'Just My Luck': 2.4224820423619167}

Мы получили ранжированный список фильмов, а также прогноз оценки, которую поставит Toby этим фильмам.

## Метрики качества для рекомендательных систем

### Качество рекомендаций

Тестирование рекомендательной системы — процесс непростой и всегда вызывающий множество вопросов, главным образом из-за неоднозначности самого понятия «качество рекомендации».

Вообще, в задачах машинного обучения есть два основных подхода к тестированию:

* offline тестирование модели на исторических данных с помощью тестов по историческим данным,
* тестирование готовой модели с помощью A/B тестирования (запускаем несколько вариантов, смотрим какой дает лучший результат).

Оба этих подхода активно применяются при разработке рекомендательных систем. Начнем с offline тестирования.

Основное ограничение, с которым приходится сталкиваться — оценить точность прогноза мы можем только на тех товарах, которые пользователь уже оценил.

Стандартный подход — это кросс-валидация методами leave-one-out и k-fold. Многократное повторение теста с усреднением результатов позволяет получить более устойчивую оценку качества.

leave-one-out — модель обучается на всех оцененных пользователем объектах, кроме одного, а тестируется на этом одном объекте. Так делается для всех n объектов, и среди полученных n оценок качества вычисляется среднее.
k-fold — выборка разбивается на k блоков одинакового размера, обучение запускается k раз так, чтобы каждый блок по одному разу попал в test и k-1 раз в train.

Все метрики качества можно условно разделить на три категории:
* Prediction Accuracy — оценивают точность предсказываемого рейтинга (похожи на метрики регрессии),
* Decision support — оценивают релевантность рекомендаций (похожи на метрики классификации),
* Rank Accuracy метрики — оценивают качество ранжирования выдаваемых рекомендаций.

Когда рейтинги оцениваются по непрерывной шкале (0-10), как правило, достаточно метрик класса Prediction Accuracy.

Формула для средней квадратичной ошибки:

$$MSE =  \frac{1}{\ell} \sum\limits_{i=1}^\ell (\hat y^{(i)}  - y^{(i)})^2 $$
$\ell$ -количество объектов.

$$RMSE = \sqrt {MSE}$$

Формула для абсолютной средней ошибке:
$$MAE =  \frac{1}{\ell} \sum\limits_{i=1}^\ell |\hat y^{(i)}  - y^{(i)}| $$


Метрики класса Decision Support работают с бинарными данными (0 и 1, да и нет). Если в нашей задаче рейтинги изначально откладываются по непрерывной шкале, их можно перевести в бинарный формат, применив решающее правило — скажем, если оценка меньше 3.5, считаем оценку «плохой», а если больше, то «хорошей».

**Таблица сопряжённости** (матрица неточности, или Confusion matrix) содержит сводные показатели качества работы классификатора. **Строки** этой таблицы соответствуют **фактическим классам** тестового набора, а **столбцы** - **предсказанным** классификатором меткам.

Таблица содержит четыре сводных показателя, каждый из которых отражает количество объектов в одной и четырех
категорий:
* **истинно позитивный** (*True positive*, **TP**) -- объект
класса `1` был верно помечен меткой `1`;
* **ложно позитивный** (*False positive*, **FP**) -- объект
фактически принадлежит классу `0`, но помечен меткой `1`;
* **истинно отрицательный** (*True negative*, **TN**) -- классификатор
верно определил, что объект класса `0` принадлежит классу `0`;
* **ложно отрицательный** (*False negative*, **FN**) -- классификатор
пометил объект меткой `0`, однако на самом деле объект принадлежит классу `1`.

Метрики:

*  $Precision = \frac{TP}{TP + FP}$, доля рекомендаций, понравившихся пользователю.
*  $Recall = \frac{TP}{TP + FN}$, доля интересных пользователю товаров, которая показана.

Как правило, рекомендации выводятся списком из нескольких позиций (сначала топовая, затем в порядке убывания приоритета). Метрики класса Rank Accuracy измеряют насколько правилен порядок показа рекомендаций в отсортированном списке.

Промежуточное положение между классами Decision Support и  Decision Support занимают метрики:

* $Precision@k$ - метрика Precision, посчитанная на Top-k записях
* $Recall@k$ - метрика Recall, посчитанная на Top-k записях
* $AverageP$ - среднее значение Precision на всем списке рекомендаций

$$ Precision@k = \frac {1}{k} \sum^k_{i=1} Relevance@i$$

$Relevance$ здесь принимает значения 0 (рекомендация хорошая) или 1 (рекомендация плохая).

Модифицированная метрика: средняя точность (англ. average precision):

$$ AP@k = \frac {\sum^k_{i=1} (Relevance@i * Precision@i)}{\sum^k_{i=1} Relevance@i}$$

Очень часто в качестве метрик качества рекомендаций  используют $MAP@k$, которая является усреднением $AP@k$

$$ MAP@k = \frac {1}{|Users|} \sum_{u} AP@k(u) $$

В случае, когда оценки релевантности могут быть вещественными (в этом смысле будем обозначать не relevance, а rating), используется другой подход к измерению качества ранжирования — дисконтированный совокупный доход (англ. discounted cumulative gain) или DCG.

$$ DCG@k = \sum^k_{i=1} \frac {2^{Rating@i}-1}{\log {(i+1)}} $$

Знаменатель зависит от позиции рекомендации, он штрафует за то, где она находится. Если рекомендация очень релевантна (высокий рейтинг), но занимает низкую позицию, то штраф будет большим. Эта метрика достигает максимума, если все релевантные рекомендации находятся в топе списка, причём отсортированные по значению рейтинга. Данную метрику принято нормировать (После нормировки метрика принимает значения от 0 до 1).

$$ nDCG@k =  \frac {DCG@k}{max(DCG@k)} $$

$max(DCG@k)$ - значение DCG при идеальном ранжировании.

https://github.com/statisticianinstilettos/recmetrics - библиотека с метриками для рекомендаций

В дополнение статья про метрики ранжирования: https://habr.com/ru/company/econtenta/blog/303458/

Для оценки качества работы рекомендательных систем можно рассмотреть две специальные метрики **Hit Rate** (HR) и **Mean Reciprocal Rank** (MRR).

Определим как **попадание** (**hit**) - ситуацию, когда фильм был рекомендован пользователю в топ-N и пользователь посмотрел его.

**Hit Rate (HR)**

**Hit Rate (процент попаданий)** определяется как общее число попаданий, нормированное на количество пользователей.

$$
\text{HR} = \frac{1}{\text{# test users}} \sum_{\text{test users}}{\text{hit}}, \quad
$$

Чем больше процент попаданий, тем лучше будет наша система рекомендаций.

*Замечание*: в некотором смысле, Hit Rate представляет собой производное от классических ML метрик precision и recall.

**Mean Reciprocal Rank (MRR)**

Отличие метрики **Mean Reciprocal Rank** от предыдущей в том, что мы учитываем не общее количество попаданий, а обратное рангу (месту в списке рекомендаций) **первое попадание**. Например, если фильм, просмотренный пользователем (попадание), стоял на втором месте, обратное рангу будет равно $\frac{1}{2}$, а если бы стоял на третьем - $\frac{1}{3}$ и так далее.

$$
\text{MRR} = \frac{1}{\text{# test users}} \sum_{\text{test users}}{\frac{1}{\text{hit rank}}}
$$

Рассмотрим на примере. Toby получил следующий список рекомендаций:

In [ ]:
getRecommendations(critics,'Toby', sim_pearson)

Допустим, после этого Toby посмотрел 5 фильмов и выставил им следующие оценки:


|Score|Film|
|----|--------------------|
|4.5 | The Departed |
|4   | The Night Listener |
|3.9 | Lady in the Water |
|3.7 | The Firm |
|3   | Just My Luck |

Как мы видим, только два фильма из списка рекомендаций попали в личный топ-3 Toby. Получается, что **попадание (hits)** у него 2.

**HR = $\dfrac{1}{1} * 2 = 2$**, так как выборка тестовых пользователей (test\_users) состоит из одного Toby.  

* **$\frac{1}{hit\_rank}$** для фильма The Night Listener - это $\dfrac{1}{2}$
* **$\frac{1}{hit\_rank}$** для фильма Lady in the Water - это $\dfrac{1}{3}$

Но фильм **The Night Listener** стоит выше в рейтинге Toby, чем **Lady in the Water**, поэтому:

**MRR = $\dfrac{1}{1}*\dfrac{1}{2} = \dfrac{1}{2}$**, так как, опять же, выборка тестовых пользователей (test\_users) состоит из одного Toby.  

## Коллаборативная фильтрация по сходству объектов (Item-based collaborative filtering)

Мы реализовали механизм выдачи рекомендаций таким образом, что для создания набора данных необходимы оценки, выставленные каждым
пользователем. Для нескольких тысяч людей это, возможно, и будет работать, но, когда речь идет о миллионах пользователей и товаров, сравнение каждого пользователя со всеми другими, а затем сравнение товаров, займет недопустимо много времени.

Техника, которую мы применяли до сих пор, называется **коллаборативной фильтрацией по схожести пользователей**. Альтернатива известна под названием **коллаборативная фильтрация по схожести образцов**.

Основная идея процедуры фильтрации по схожести образцов заключается в том, чтобы для каждого образца заранее вычислить большинство похожих на него. Тогда для выдачи рекомендаций алгоритму достаточно будет найти те образцы, которым пользователь выставил наивысшие оценки, а затем сформировать список образцов, максимально похожих на эти.

Отметим одно существенное отличие: хотя на первом шаге необходимо исследовать все данные, результаты сравнения образцов изменяются не так часто, как результаты сравнения пользователей.

Чтобы сравнивать образцы, нужно первым делом построить полный набор данных о похожих образцах. В первую очередь, нам нужно переделать словарь предпочтений критиков таким образом, чтобы теперь был словарь фильмов и их оценок:
```python
film_dict = {
    'film_name_1':
    {'critic_1': 'score_1',
     'critic_2': 'score_2',
     ...
    },
    ...
}
```
Далее для каждого фильма посчитать наиболее похожие образцы и коэффициенты похожести для них. Для подсчета схожести будем использовать евклидово расстояние.



In [ ]:
# обращаем матрицу предпочтений, чтобы строки соответствовали образцам
def transformPrefs(critics_dict):

    # инициилизируем новый словарь с фильмами и их оценками
    result = {}

    # для каждого критика в словаре
    for person in critics_dict:

        # для каждого оцененного им фильма
        for item in critics_dict[person]:

            # добавляем такой объект в словарь
            result.setdefault(item, {})
            # меняеи местами человека и предмет
            result[item][person] = critics_dict[person][item]
    return result

In [ ]:
pp.pprint (transformPrefs (critics))

{   'Just My Luck': {   'Claudia Puig': 3.0,
                        'Gene Seymour': 1.5,
                        'Lisa Rose': 3.0,
                        'Mick LaSalle': 2.0},
    'Lady in the Water': {   'Gene Seymour': 3.0,
                             'Jack Matthews': 3.0,
                             'Lisa Rose': 2.5,
                             'Michael Phillips': 2.5,
                             'Mick LaSalle': 3.0},
    'Snakes on a Plane': {   'Claudia Puig': 3.5,
                             'Gene Seymour': 3.5,
                             'Jack Matthews': 4.0,
                             'Lisa Rose': 3.5,
                             'Michael Phillips': 3.0,
                             'Mick LaSalle': 4.0,
                             'Toby': 4.5},
    'Superman Returns': {   'Claudia Puig': 4.0,
                            'Gene Seymour': 5.0,
                            'Jack Matthews': 5.0,
                            'Lisa Rose': 3.5,
                            'M

**Задание 1.1** Реализуйте функцию **calculateSimilarItems()**, которая делает сразу оба шага. На вход её нужно подать словарь с оценками критиков *critics* и число образцов, которые мы считаем наиболее похожими (остальные не выводим). Например, так `calculateSimilarItems(critics, n=10)`

**Code 16:**

In [ ]:
# Создать словарь, содержащий для каждого образца те образцы, которые больше всего похожи на него.
def calculateSimilarItems(prefs, n=10):
    films = transformPrefs(critics)
    sim_films = dict()
    for film in films:
        sim_films[film] = topMatches(films, film, n, sim_distance)

    return sim_films

In [ ]:
itemsim = calculateSimilarItems(critics, n=3)

In [ ]:
pp.pprint(itemsim)

{   'Just My Luck': [   'Lady in the Water',
                        'You, Me and Dupree',
                        'The Night Listener'],
    'Lady in the Water': [   'You, Me and Dupree',
                             'The Night Listener',
                             'Snakes on a Plane'],
    'Snakes on a Plane': [   'Lady in the Water',
                             'The Night Listener',
                             'Superman Returns'],
    'Superman Returns': [   'Snakes on a Plane',
                            'The Night Listener',
                            'Lady in the Water'],
    'The Night Listener': [   'Lady in the Water',
                              'Snakes on a Plane',
                              'Just My Luck'],
    'You, Me and Dupree': [   'Lady in the Water',
                              'Just My Luck',
                              'The Night Listener']}


Получили вложенный словарь с фильмами и их похожестью на другие фильмы.

Теперь мы готовы выдавать рекомендации, пользуясь словарем данных о схожести образцов. По аналогии с тем, что мы уже делали, необходимо получить список всех образцов, которым пользователь выставлял оценки, найти похожие и взвесить их с учетом коэффициентов подобия (умножить оценку на коэффициент подобия).

Реализуйте этот механихм в функции **getRecommendedItems()**. На вход этой функци подается:
* словарь с оценками пользователя (в нашем случае это словарь критиков, *critics*)
* словарь с данными о схожести образцов, *itemsim*
* имя критика, для которого строим рекомендации

Код, для того, чтобы получить рекомендации для пользователя Toby `getRecommendedItems(critics, itemsim, 'Toby')`

**Code 17:**

In [ ]:
def getRecommendedItems(prefs, itemMatch, user):
    # Берем фильмы, которые оценил пользователь
    user_films = prefs[user]
    # Берем фильмы и их оценки критиков
    films = transformPrefs(prefs)
    # Инициализируем словарь рек. фильмов с их предполагаемыми оценками,
    # которые похожи на те что пользователь уже смотрел
    rankings = dict()
    for film in user_films:
        max_wscore = 0
        rec_film = ''
        # Берем оцененный фильм и похожий фильм
        for sim_film in itemMatch[film]:
            # Если фильм оцененный пользователем, то пропускаем
            if sim_film in user_films:
                continue

            # Считаем взвешанный коэф. подобия
            score = sim_distance(films, film, sim_film)
            wscore = user_films[film] * score
            # Если взвешанная схожесть максимальна, то запоминаем
            if wscore > max_wscore:
                max_wscore = wscore
                rec_film = sim_film

        # Если схожесть есть, то сохраняем в рекомендации
        if max_wscore:
            rankings[rec_film] = user_films[film] * max_wscore

    # Сортируем рекомендации
    return dict(sorted(rankings.items(), reverse=True, key=lambda x: x[1]))

In [ ]:
rec_films = getRecommendedItems(critics, itemsim, 'Toby')
rec_films

{'The Night Listener': 4.042404937393152,
 'Lady in the Water': 0.4494897427831781}

Рекомендация получилась так себе. Ведь это просто ранжированный список фильмов, которые он не смотрел.


**Задание 1.2**

Давайте в рекомендацию будем включать только те фильмы, за которые прогнозируемый средний балл превышает средний балл за те фильмы, которые уже оценил критик.

In [ ]:
mean_score = np.mean(list(critics['Toby'].values()))
roi_rec_films = {name: rec_films[name] for name in rec_films if rec_films[name] > mean_score}
roi_rec_films

{'The Night Listener': 4.042404937393152}